In [1]:
import osmnx
from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap
import osmread
from dataclasses import dataclass
from geopandas.geodataframe import GeoDataFrame
import json
from dataclasses import dataclass
import pandas as pd
import numpy as np
import folium
from typing import Iterable
import networkx as nx
from datetime import datetime, timedelta
import numba
from scipy.interpolate import interp1d
from typing import Iterable
import networkx as nx
from datetime import datetime, timedelta
import matplotlib as mpl
from folium.features import DivIcon
from helper_utils import MatchedRoutes,Segments

# f = open('../matched-results-1500_gh.json')
# data = json.load(f)


graph = osmnx.graph_from_place('Porto, Portugal', network_type='drive', simplify=False,which_result=2)
graph_proj = osmnx.project_graph(graph)
nodes_proj, edges_proj = osmnx.graph_to_gdfs(graph_proj, nodes=True, edges=True)


map_con = InMemMap("myosm", use_latlon=True, use_rtree=True, index_edges=True)
for nid, row in nodes_proj[['lat', 'lon']].iterrows():
    map_con.add_node(nid, (row['lat'], row['lon']))
for eid, _ in edges_proj.iterrows():
    map_con.add_edge(eid[0], eid[1])
    
matcher = DistanceMatcher(map_con,
                         max_dist=1000, max_dist_init=25,  # meter
                         min_prob_norm=0.001,
                         non_emitting_length_factor=0.75,
                         obs_noise=50, obs_noise_ne=75,  # meter
                         dist_noise=50,  # meter
                         non_emitting_states=True,
                         max_lattice_width=5)


# df = df[df['MATCHED_RESULTS'].apply(lambda x: len(x['Matched_edge_for_each_point'])>5)]

# Task 1

In [2]:
df = pd.read_parquet('./1500_trips.parquet')
df

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1495,1372672434620000051,B,NaN,44.0,20000051,1372672434,A,False,"[[-8.625447,41.149332],[-8.625429,41.149323],[..."
1496,1372674326620000557,B,NaN,13.0,20000557,1372674326,A,False,"[[-8.62839,41.15718],[-8.628183,41.15736],[-8...."
1497,1372672055620000653,B,NaN,18.0,20000653,1372672055,A,False,"[[-8.619777,41.148018],[-8.620056,41.147793],[..."
1498,1372673543620000598,C,NaN,NaN,20000598,1372673543,A,False,"[[-8.58654,41.148999],[-8.587233,41.149125],[-..."


In [3]:
from dataclasses import dataclass
from geopandas.geodataframe import GeoDataFrame
@dataclass
class MatchedRoutes:
    gps_coords:np.ndarray
    states: list[tuple[int,int]]
    df: GeoDataFrame
    
    
    @staticmethod
    def from_state(polyline: str,
                   matcher: DistanceMatcher,
                   nodes_proj: GeoDataFrame):
        gps_coords = np.asarray(json.loads(polyline)).reshape(-1,2)
        if len(gps_coords)>0:
            gps_coords = gps_coords[:,[1,0]]
            states, lastidx = matcher.match(gps_coords)
            df = nodes_proj.loc[[s[0] for s in states]]
            return MatchedRoutes(gps_coords,states,df)
    
test = MatchedRoutes.from_state(df['POLYLINE'].values[0],matcher,nodes_proj)

In [4]:
matcher = DistanceMatcher(map_con,
                         max_dist=10000, max_dist_init=50,  # meter
                         min_prob_norm=0.001,
                         non_emitting_length_factor=0.75,
                         obs_noise=25, obs_noise_ne=25,  # meter
                         dist_noise=25,  # meter
                         non_emitting_states=True)
                         #max_lattice_width=250,max_lattice_height=250)

In [5]:
from tqdm.notebook import tqdm
from multiprocessing import Pool
from functools import partial

# fn = partial(MatchedRoutes.from_state, matcher=matcher,nodes_proj=nodes_proj)
# partial(func, b=second_arg)
# with Pool(4) as pool:
    # result = pool.map(fn,df['POLYLINE'].values)
    # result =  list(tqdm(pool.imap(fn, df['POLYLINE'].values), total=len(df['POLYLINE'].values)))
    
routes = []
for polyline in tqdm(df['POLYLINE'].values[:15]):
    routes.append(MatchedRoutes.from_state(polyline,matcher,nodes_proj))



  0%|          | 0/15 [00:00<?, ?it/s]

# Task 2,3 and 4: Plot GPS Points + Map Matching

In [10]:
import seaborn as sns
pal = sns.color_palette('pastel').as_hex()

m = folium.Map(location=routes[0].gps_coords.mean(0), zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(m)

marker_colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]
for i,r in enumerate(routes[:15]):
    # c = pal[i%len(pal)]
    c = marker_colors[i]
    
    for loc in r.gps_coords:
        icon=folium.Icon(color=c)
        folium.Marker( location=loc,icon=icon , radius=3 ).add_to(m)
        
m

In [7]:
# import seaborn as sns
pal = sns.color_palette('pastel').as_hex()

m = folium.Map(location=routes[0].gps_coords.mean(0), zoom_start=15)
folium.TileLayer('cartodbpositron').add_to(m)

# folium.PolyLine(test.df[['lat','lon']].values,
#                 color='blue',
#                 weight=5,
#                 opacity=0.8).add_to(m)
for i,r in enumerate(routes[:15]):
    c = pal[i%len(pal)]
    # for loc in r.gps_coords:
        # folium.Marker( location=loc, fill_color=c, radius=3 ).add_to(m)
    folium.PolyLine(r.gps_coords,
                color=c,
                weight=5,
                opacity=0.8).add_to(m)
    if len(r.df)>0:
        folium.PolyLine(r.df[['lat','lon']].values,
                    color='blue',
                    weight=5,
                    opacity=0.8).add_to(m)
m